# Introduction: 
Dans ce projet, on va comprendre comment Google classe les pages web selon un indice dit de popularité (PageRank). C'est grâce à cette technique que Google est devenu l'un des moteurs de recherches les plus connus au monde. On étudiera plus précisément les propriétés mathématiques cachées derrière ce procédé.
Comme son nom l'indique, l'indice de popularité mesure quantitativement l'importance et la popularité d'une page web. On commencera par introduire des fonctions pour le calcul de la matrice des hyperliens $H$ et la matrice de Google $G=\alpha S+ (1-\alpha)ev'$, où $S$ est une matrice stochastique déduite de $H$ en remplaçant les lignes de zéros de $H$ par le vecteur ligne $(\frac{1}{n},...,\frac{1}{n})$, et $v$ un vecteur stochastique. on implémentera par la suite l'algorithme de la puissance itérée, qui est basé sur un corollaire du théorème de Perron-Frobenius afin de calculer l'indice de popularité. Par ailleurs on implémentera une fonction $PageRank$ qui jouera un rôle important pour la rapidité du calcul de PageRank. On verra après l'importance du vecteur $v$ dans le calcul du PageRank, et comment Google peut améliorer le classement d'une page dont l'indice de popularité est faible en modifiant $v$.
Notons que tous les algorithmes ont été faits en $Python$.

In [37]:
import numpy as np
import scipy as sp
from scipy import linalg as la
from scipy import sparse
from copy import deepcopy


## question1

Voici la fonction qui permet de calculer la matrice stochastique $S$ :

In [38]:
#Fonction pour calculer S à partir de H
def MatriceS(H):
     '''Entrée : la matrice des hyperliens
        Sortie : la matrice S, en remplaçant les lignes 0 de H par 1/n'''
     n=np.shape(H)[0] # H est une matrice n*n
     S=deepcopy(H)
     u=np.ones(n)/n #vecteur dont les composantes sont 1/n
     L=[0]*n
     i=0 
     for ligne in H:
          if list(ligne) == L:
               S[i]=u
          i+=1
     return S

Exemple: On prend $$H=\begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 1/2 & 1/2 \\ 0 & 1/2 & 0 & 1/2 \\ 0 & 0 & 0 & 0 \end{bmatrix}$$, sa matrice S est:

In [64]:
H=np.array([[0,0,1,0],[0,0,1/2,1/2],[0,1/2,0,1/2],[0,0,0,0]])
print(MatriceS(H))

[[0.   0.   1.   0.  ]
 [0.   0.   0.5  0.5 ]
 [0.   0.5  0.   0.5 ]
 [0.25 0.25 0.25 0.25]]


Voici la fonction qui permet de calculer la matrice de Google $G$ :

In [39]:
#Matrice Google
def GoogleMatrice(H, alpha, v):
     '''Entrée : la matrice des hyperliens H, un réel alpha et un vecteur v
        Sortie : la matrice G = alpha*S+(1-alpha)e'v'''
     n=np.shape(H)[0]
     S=MatriceS(H)
     e=np.ones((n,1))
     v=v.reshape((1,n))
     G=alpha*S+(1-alpha)*np.dot(e,v)
     return G



Exemple: La matrice de Google $G$ de $H$ (définie précédemment):

In [66]:

print(GoogleMatrice(H, 0.85, np.array([0.25,0.25,0.25,0.25])))

[[0.0375 0.0375 0.8875 0.0375]
 [0.0375 0.0375 0.4625 0.4625]
 [0.0375 0.4625 0.0375 0.4625]
 [0.25   0.25   0.25   0.25  ]]


## question 2

In [69]:
def puissanceiteree(A,x0,epsilon,nmax):
    '''Entrée: Matrice A, vecteur x0, précision epsilon, nombre d'itération maximal nmax
    Sortie: vecteur x et k nombre d'itérations '''
    z=np.dot(x0,A)
    etha=la.norm(z-x0, 1)
    x=z
    k=1
    while etha > epsilon and k<nmax:
        z=np.dot(x,A)
        etha=la.norm(z-x, 1)
        x=z
        k+=1 
    return x,k

## question 3

In [70]:
epsilon=0.00001
nmax=1000
#On prend alpha= 0.85 et v=(1/n)*e
n=n=np.shape(H)[0]
alpha=0.85
v=np.ones(n)/n
print("La matrice H :")
print(H)
print("la matrice S :")
print(MatriceS(H))
print("La matrice de Google associée :")
G=GoogleMatrice(H,alpha,v)
print(G)
print("Le calcul du PageRank")
Pi,k=puissanceiteree(G,v,0.01,100)
print(f"le pageRank est pi={Pi}")
print(f"La méthode converge en {k} itérations")

print("Testons la méthode: on calcul pi*G, normalement on doit trouver pi")
Pi=Pi.reshape((1,n))
print(f"pi*G={np.dot(Pi,G)}")

La matrice H :
[[0.  0.  1.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.5 0.  0.5]
 [0.  0.  0.  0. ]]
la matrice S :
[[0.   0.   1.   0.  ]
 [0.   0.   0.5  0.5 ]
 [0.   0.5  0.   0.5 ]
 [0.25 0.25 0.25 0.25]]
La matrice de Google associée :
[[0.0375 0.0375 0.8875 0.0375]
 [0.0375 0.0375 0.4625 0.4625]
 [0.0375 0.4625 0.0375 0.4625]
 [0.25   0.25   0.25   0.25  ]]
Le calcul du PageRank
le pageRank est pi=[0.11040664 0.24134933 0.30540718 0.34283685]
La méthode converge en 6 itérations
Testons la méthode: on calcul pi*G, normalement on doit trouver pi
pi*G=[[0.11035283 0.24015088 0.30677194 0.34272435]]


## question 4

On va tester sur les deux vecteurs $v$ différents:

In [74]:

#Pour la tolérance epsilon=10^{-2}, tout va bien !!!
v1=np.array([0.1,0.4,0.1,0.4])
v2=np.array([0.02,0.48,0.02,0.48])
G1=GoogleMatrice(H,0.85,v1)
G2=GoogleMatrice(H,0.85,v2)
print(f"le pageRank pour le vecteur {v1} est :")
print(puissanceiteree(G1,v,0.01,100))
print(f"le pageRank pour le vecteur {v2} est :")
print(puissanceiteree(G2,v,0.01,100))

le pageRank pour le vecteur [0.1 0.4 0.1 0.4] est :
(array([0.09315082, 0.25860515, 0.28079692, 0.36744711]), 6)
le pageRank pour le vecteur [0.02 0.48 0.02 0.48] est :
(array([0.08394772, 0.26780825, 0.26767145, 0.38057258]), 6)


On remarque que 
l'indice de popularité des pages p1 et p3  s'est diminué pour $v_2$, alors que pour p2 et p4 l'indice augmente,
on en déduit que si on veut augmenter l'indice de popularité d'une page, il suffit d'augmenter le coefficient correspondant de $v$.

## question 5

In [76]:
def PageRank(H,v,alpha,pi0,epsilon,nmax):
    '''Entrée: Matrice H, vecteur v, nombre alpha, vecteur pi0, précision epsilon, nombre d'itération nmax
       Sortie: vecteur pi, nombre d'itération k'''
    H=sparse.coo_matrix(H)
    n=H.shape[0]
    v=v.reshape((1,n))
    e=np.ones((1,n))/n
    L=[0]*n
    i=0
     #construction du vecteur a
    a=np.zeros(n)
    for ligne in H.toarray():
        if list(ligne)==L:
              a[i]=1
        i+=1
    pi0=pi0.reshape((1,n))
    z=alpha*(pi0@H+(pi0.reshape((n,))@a)*e)+(1-alpha)*v
    etha=la.norm(z-pi0, 1)
    pi=z
    k=1
    while etha > epsilon and k<nmax:
        z=alpha*(pi@H+(pi.reshape((n,))@a)*e)+(1-alpha)*v
        etha=la.norm(z-pi, 1)
        pi=z
        k+=1
    return pi,k


In [77]:
pi0=np.ones(n)/n
#Test sur les exemples précédents

print(f"le pageRank pour le vecteur {v1} est {PageRank(H,v1,alpha,pi0,0.01,nmax)[0]} en {PageRank(H,v1,alpha,pi0,0.01,nmax)[1]} itérations")
print(f"le pageRank pour le vecteur {v2} est {PageRank(H,v2,alpha,pi0,0.01,nmax)[0]} en {PageRank(H,v2,alpha,pi0,0.01,nmax)[1]} itérations")

le pageRank pour le vecteur [0.1 0.4 0.1 0.4] est [[0.09271074 0.25609874 0.28342194 0.36776857]] en 5 itérations
le pageRank pour le vecteur [0.02 0.48 0.02 0.48] est [[0.08348164 0.26532784 0.27026007 0.38093044]] en 5 itérations


On remarque que on retrouve les mêmes résultats qu'auparavant avec un nombre d'itération plus petit.

## question 6

La fonction ci-dessous génère une matrice H aléatoirement. Voici comment on procède:\
D'abord on crée une matrice aléatoire $m*m$ remplie par $0$ et $1$. Ensuite, on ajoute des $0$ pour obtenir une matrice $n*n$. Et on permute aléatoirement les colonnes, puis on normalise toutes les lignes. A la fin, on stocke notre matrice en mode sparse.

In [78]:
def genererH(n,m):
     '''entrée : deux entier n et m
        Sortie : La matrice H '''
     M1=np.random.randint(0,2,size=(m,m)) #on crée une matrice m*m aléatoirement
     M2=np.zeros((m,n-m)) 
     M3=np.zeros((n-m,n))
     I=np.vstack((np.hstack((M1,M2)),M3)) #là on obtient une matrice dont les éléments en haut à gauche (m*m) des 0 ou 1 aléatoirement, et des 0 ailleurs
     I=I[:, np.random.permutation(I.shape[0])] #on permute les colonnes aléatoirement, on obtient donc une matrice n*n avec dans chaque ligne au plus m coefficient=1
     s=I.sum(axis=1) #somme sur les lignes dans le but de normaliser chaque ligne
     s[s==0]=1
     H=np.transpose(np.transpose(I)/s)
     H=sparse.coo_matrix(H)
     return H

Exemple d'une matrice $H$ générée par la fonction genererH avec $n=5$ et $m=3$
<bt>
Voici la matrice $H$ en mode sparse :

In [82]:
H=genererH(5,3)
print(H)

print("voici la même matrice en mode standard")
print(H.toarray())

  (0, 1)	0.5
  (0, 2)	0.5
  (2, 1)	0.5
  (2, 2)	0.5
voici la même matrice en mode standard
[[0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]]


In [48]:
alpha=0.5
epsilon=0.001
import time

## question 7

On reprend les mêmes fonctions pour le calcul de $G$, mais cette fois-ci en mode sparse.

In [83]:
def MatriceS_sparse(H):
     '''Entrée : la matrice des hyperliens
        Sortie : la matrice S, en remplaçant les lignes 0 de H par 1/n'''
     n=np.shape(H)[0] # H est une matrice n*n
     S=deepcopy(H)
     S=S.tolil()
     u=np.ones(n)/n #vecteur dont les composantes sont 1/n
     L=[[0]*n]
     i=0 
     for ligne in H.tolil():
          if ligne.toarray().tolist() == L:
               S[i,:]=u
          i+=1
     return S

In [84]:
def GoogleMatrice_sparse(H, alpha, v):
     '''Entrée : la matrice des hyperliens H, un réel alpha et un vecteur v
        Sortie : la matrice G = alpha*S+(1-alpha)e'v'''
     n=np.shape(H)[0]
     S=MatriceS_sparse(H)
     e=np.ones((n,1))
     v=v.reshape((1,n))
     G=alpha*S+(1-alpha)*np.dot(e,v)
     return G


A l'aide de genererH, on va créer une matrice d'hyperliens H génerée aléatoirement avec $n=1000$ et $m=25$.

NB:Le cas de $n=10000$ et $m=50$ sera traité ultérieurement, maintenant on va se contenter de comparer la puissance itérée et PageRank pour $n=1000$ et $m=25$ .

In [86]:
n=1000
m=25
v=np.ones(n)/n
H=genererH(n,m)
pi0=np.ones(n)/n 

On va comparer le temps de calcul entre les deux méthodes:

In [87]:
#Avec la puissance itérée
t1=time.time()
G=GoogleMatrice_sparse(H, 0.05, v)
puissanceit=puissanceiteree(G,pi0,0.001,nmax)
t2=time.time()
print(f"Par la puissance itérée, le calcul se fait en {t2-t1} s")
print(f"ça donne : pi ={puissanceit[0]}")

Par la puissance itérée, le calcul se fait en 11.389495134353638 s
ça donne : pi =[[0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00103607 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875

In [88]:
#Avec le PageRank
t3=time.time()
pagerank=PageRank(H,v,0.05,pi0,0.001,nmax)
t4=time.time()
print(f"Par pageRank, le calcul se fait en {t4-t3} s")
print(f"ça donne : pi ={pagerank[0]}")

Par pageRank, le calcul se fait en 1.4959990978240967 s
ça donne : pi =[[0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00103607 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875

Remarque: La méthode de PageRank est plus rapide que celle de la puissance itérée.

On teste le PageRank pour $n=10000$ et $m=50$:

In [91]:
#Avec le PageRank
n2=10000
m2=50
v2=np.ones(n2)/n2
H2=genererH(n2,m2)
pi01=np.ones(n2)/n2
t3=time.time()
pagerank=PageRank(H2,v2,0.05,pi01,0.001,nmax)
t4=time.time()
print(f"Par pageRank, le calcul se fait en {t4-t3} s")
print(f"ça donne : pi ={pagerank[0]}")

Par pageRank, le calcul se fait en 139.65424847602844 s
ça donne : pi =[[9.9975e-05 9.9975e-05 9.9975e-05 ... 9.9975e-05 9.9975e-05 9.9975e-05]]


## question 9


<bt>Un commerçant nous propose une grosse somme d'argent pour que ses pages soient bien classées. Il a $5$ pages : $P_6,...P_{10}$, on cherche donc une procédure 
pour rendre ses pages bien classées : il faudrait donc que les indices de popularités de ses pages soient grands, or on a vu que quand on gonfle un élément 
du vecteur $v$, l'indice de popularité correspondant augmente. On joue alors sur les composantes $v_6,...,v_{10}$ du vecteur $v$. Voici comment on va procéder :\
On se donne $\epsilon >0$ et soit $v=(v_1,...,v_n)$ un vecteur stochastique, on ajoutera $\epsilon$ à $v_i$ avec $i \in \{6,...,{10}\}$. Rappelons que le vecteur $v$
 est stochastique, c'est-à-dire il faut que la somme de ses éléments soit égale à 1 et que toutes ses composantes sont positives.\
On pose $v'=(v'_1,...,v'_n)$ tel que :\
<bt>$v'_i = v_i + \epsilon$ si $i \in \{6,...,{10}\}$\
et $v'_i = v_i-\frac{5\epsilon}{n-5}$ sinon.\
Puisque la somme des $v_i$ égale à 1, donc le vecteur $v'$ sera stochastique aussi, la somme de ses composantes égale à bien 1. C'est ce vecteur que l'on utilisera pour faire plaisir au commerçant. Voici l'algorithme du calcul de $v'$:

In [57]:
def Nouveau_vec(v):
     """Entrée : vecteur v
        Sortie vecteur x, qui représentera v' """
     n=len(v)
     e=0.01 
     x=np.ones(n)
     for i in range(0,len(v)):
          if i in [5,6,7,8,9]:
               x[i]=v[i]+e
          else:
               x[i]=v[i]-5*e/(n-5)
     return x
print("On prend maintenant n=10000 et m=50")
print("Voici le PageRank avant le contrat")
p1=PageRank(H,v,0.05,pi0,0.001,nmax)[0]
print("p = ",p1)
print("Voici le PageRank après le contrat")
p2=PageRank(H,Nouveau_vec(v),0.05,pi0,0.001,nmax)[0]
print("p = ",p2)

print("Voici les indices de popularités des pages du commerçant avant le contrat:")
print(p1[0][5:10])
print("Les voici après après le contrat:")
print(p2[0][5:10])

On prend maintenant n=10000 et m=50
Voici le PageRank avant le contrat
p =  [[0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00105411 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00105448 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.00099875 0.00099875 0.00099875
  0.00099875 0.00099875 0.00099875 0.000

p =  [[0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.01049642
  0.01049642 0.01049642 0.01049642 0.01049642 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00108899 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00107691 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00094868 0.00094868 0.00094868 0.00094868 0.00094868
  0.00094868 0.00114088 0.00094868 0.000948